## IMPORTAR MODULOS, LECTURA DEL ARCHIVO Y TIPO DE COLUMNAS

In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

#LINK DE DESCARGA DEL CSV
#https://drive.google.com/file/d/15Ofxvl8CbGrq2BjwmE7GjRCMR_YixKb4/view
data_location = "./properati.csv"

data = pd.read_csv(data_location, sep=",")
df = pd.read_csv(data_location, sep=",")

df.drop(['operation', 'Unnamed: 0','geonames_id','lat-lon','lat','lon','properati_url','image_thumbnail', 'price_aprox_local_currency'], axis=1, inplace=True)
df.dtypes

## Cantidad de nulos iniciales en el dataframe

In [ ]:
df.isnull().sum()

## Inicio de Etapa de Busqueda de Datos

### Búsqueda de Precios y Monedas en title

In [ ]:
price_pattern = "(?P<currency>U?\$[SD]?|pesos|ARS|USD|dolares)\s*?(?P<price>\d{1,3}(?:[.,]?\d*)*(?:[.,]?\d{2})?)"
price_regex =  re.compile(price_pattern)

price_match = df.title.apply(lambda x: x if x is np.NaN else price_regex.search(x))
price_match_mask = price_match.notnull()
df.loc[price_match_mask, "PriceTitle"] = price_match[price_match_mask].apply(lambda x: x.group("price"))
df.loc[price_match_mask, "CurrencyTitle"] = price_match[price_match_mask].apply(lambda x: x.group("currency"))

df["PriceTitle"] = df["PriceTitle"].apply(lambda x: x if (type(x) != str) else x.replace(".", ""))
df["PriceTitle"] = df["PriceTitle"].apply(lambda x: x if (type(x) != str) else x.replace(",", ""))
df["PriceTitle"] = df["PriceTitle"].astype(float)


### Búsqueda de M2 en description

In [ ]:
m2_pattern = "\s(?P<metros>\d{0,3}?[.]?\d*)\s?(?P<sufijo>m2|M2|metros|mts|m²)"
m2_regex =  re.compile(m2_pattern)
m2_match = df.description.apply(lambda x: x if x is np.NaN else m2_regex.search(x))
m2_match_mask = m2_match.notnull()
df.loc[m2_match_mask, "M2"] = m2_match[m2_match_mask].apply(lambda x: x.group("metros"))

### Búsqueda de ambientes en description

In [ ]:
rooms_pattern = "\s(?P<ambientes>\d\d?)(\s?)(?P<sufijo>AMB|amb|Amb)"
rooms_regex =  re.compile(rooms_pattern)

rooms_match = df.description.apply(lambda x: x if x is np.NaN else rooms_regex.search(x))
rooms_match_mask = rooms_match.notnull()
df.loc[rooms_match_mask, "Ambientes"] = rooms_match[rooms_match_mask].apply(lambda x: x.group("ambientes"))


### Búsqueda de cocheras en description

In [ ]:
garage_pattern = "(?P<garage>cochera|garage|estacionamiento)"
garage_regex =  re.compile(garage_pattern)

garage_match = df.description.apply(lambda x: x if x is np.NaN else garage_regex.search(x))
garage_match_mask = garage_match.notnull()
df.loc[garage_match_mask, "Garage"] = True
df["Garage"].fillna(False,inplace=True)


## Inicio de Etapa de Merge de Datos Obtenidos con Datos Iniciales

### Sumo los nuevos precios encontrados a la columna original y comparo la cantidad de nulos

In [ ]:
df.loc[(pd.isnull(df["price"])),"price"] = df["PriceTitle"]
print('Cantidad price null en df inicial:', data["price"].isnull().sum())
print('Cantidad price null en df trabajado:',df["price"].isnull().sum())

In [ ]:
df.loc[(pd.isnull(df["price_aprox_usd"])),"price_aprox_usd"] = df["PriceTitle"]

### Funcion para homogeneizar la columna CurrencyTitle encontradas, para después poder sumarlas a la columna currency Original

In [ ]:
def evaluate_currency(currency):
    result = 'NA'
    arrayPesos = ['PESOS', '$']
    arrayDolar = ['U$S', 'U$D', 'DOLAR', "$D", "USD", "$S", "U$"]
    if [s for s in arrayDolar if currency == s]:
        result = 'USD'
    elif [s for s in arrayPesos if currency == s]:
        result = 'ARS'
    else:
        result = np.NaN
    return result

### Aplico la función evaluate_currency a la columna CurrencyTitle 

In [ ]:
df['CurrencyTitle'] = df['CurrencyTitle'].apply(lambda x: evaluate_currency(x))

### Sumo las nuevas currency obtenidas a la columna original y comparo la cantidad de nulos

In [ ]:
df.loc[(pd.isnull(df["currency"])),"currency"] = df["CurrencyTitle"]
print('Cantidad currency null en df inicial:', data["currency"].isnull().sum())
print('Cantidad currency null en df trabajado:', df["currency"].isnull().sum())

### Completo la columna price_aprox_usd con los nuevos valores encontrados en ARS con su equivalente en USD

In [ ]:
serie_ars = df[df["currency"] == "ARS"]
mask_nulls_ars = serie_ars["PriceTitle"].notnull()
index_ars = serie_ars[mask_nulls_ars].index
df.iloc[index_ars, 7]= df["price"].apply(lambda x: round(x/17.83, 2))

### Completo los valores nulos de place_name con el valor Tigre, el mismo se obtuvo de place_with_parent_names

In [ ]:
df["place_name"].fillna(value="Tigre", inplace=True)

### Convierto la columna Ambientes a Float y completo los null de la columna rooms con los valores encontrados en Ambientes

In [ ]:
df["Ambientes"] = df["Ambientes"].astype(float)
df.loc[(pd.isnull(df["rooms"])),"rooms"] = df["Ambientes"]
print('Cantidad rooms null en df inicial:', data["rooms"].isnull().sum())
print('Cantidad rooms null en df trabajado:', df["rooms"].isnull().sum())

### Proceso para mergear en una columna los valores de surface_total_in_m2, surface_covered_in_m2 y M2

In [ ]:
df["M2"].replace(to_replace = "", value= np.NaN, inplace = True)
df["M2"] = df["M2"].astype(float)

In [ ]:
#si solo tengo valor en M2, lo llevo a metros
df['metros1'] = df[(df['surface_total_in_m2'].isnull()) & (df['surface_covered_in_m2'].isnull()) & (df['M2'].notnull())]["M2"]
#si tengo valor en surface_covered_in_m2 lo llevo a metros
df['metros2'] = df[(df['surface_total_in_m2'].isnull()) & (df['surface_covered_in_m2'].notnull())]['surface_covered_in_m2']
#si tengo valor en surface_total_in_m2 lo llevo a metros
df['metros3'] = df[(df['surface_total_in_m2'].notnull()) & (df['surface_covered_in_m2'].isnull())]['surface_total_in_m2']
#si tengo covered y total, tomo total
df['metros4'] = df[(df['surface_total_in_m2'].notnull()) & (df['surface_covered_in_m2'].notnull())]['surface_total_in_m2'] 

df["metros1"].fillna(0, inplace=True)
df["metros2"].fillna(0, inplace=True)
df["metros3"].fillna(0, inplace=True)
df["metros4"].fillna(0, inplace=True)

df['m2_calculated'] = df.apply(lambda x: x['metros1'] + x['metros2'] + x["metros3"] + x["metros4"], axis=1)

df["m2_calculated"].replace(to_replace = 0, value= np.NaN, inplace = True)

df.drop(["metros1", "metros2", "metros3", "metros4", "M2", "surface_covered_in_m2", "surface_total_in_m2"], axis=1, inplace = True)

===================================================================================================================
## Análisis
===================================================================================================================

### Cantidad de unidades agrupadas por state_name y property_type

In [ ]:
pd.pivot_table(df[['property_type','state_name']], index=['state_name'], columns=['property_type'], aggfunc=len, margins=True, margins_name='Total').fillna(0).sort_values(by='Total', ascending=False)

### Precio promedio por property_type agrupado por state_name

In [ ]:
round(pd.pivot_table(df[['property_type','state_name', 'price_aprox_usd']], index=['state_name'], columns=['property_type'], aggfunc=[np.mean]).fillna(0).sort_values(by="state_name"))

### Top provincias con precios promedios mas altos

In [ ]:
df_notnull_mask = df['price_aprox_usd'].notnull()
round(df[df_notnull_mask].groupby(['state_name'], as_index=False)['price_aprox_usd'].agg({'price_aprox_usd': 'mean'})[:10].sort_values(by='price_aprox_usd',ascending=False),2)

### Top ciudades con precios promedios mas altos

In [ ]:
df_notnull_mask = df['price_aprox_usd'].notnull()
round(df[df_notnull_mask].groupby(['place_name'], as_index=False)['price_aprox_usd'].agg({'price_aprox_usd': 'mean'})[:10].sort_values(by='price_aprox_usd',ascending=False),2)

### Cantidad de cada tipo de propiedad distribuida en los barrios mas significativos

In [ ]:
with sns.axes_style('white'):
    g = sns.catplot("state_name", data=data, aspect=2.4,hue='property_type', order = ["Capital Federal","Bs.As. G.B.A. Zona Norte",
                                                                                              "Bs.As. G.B.A. Zona Sur","Córdoba", "Bs.As. G.B.A. Zona Oeste"], kind="count")
    g.set_ylabels('cant')

### Cantidad de ambientes de cada tipo de propiedad de los barrios mas significativos

In [ ]:
with sns.axes_style('white'):
    g = sns.catplot("state_name","rooms", data=data, aspect=2.8,hue='property_type', order = ["Capital Federal","Bs.As. G.B.A. Zona Norte",
                                                                                              "Bs.As. G.B.A. Zona Sur","Córdoba", "Bs.As. G.B.A. Zona Oeste", "Neuquén"], kind="strip")
    g.set_ylabels('cant')

In [ ]:
df_mask = (df['price_aprox_usd'].notnull()) & (df['rooms'].notnull())
sns.jointplot(x="price_aprox_usd", y="rooms", data=df[df_mask])

In [ ]:
dfz = round(df.groupby(['state_name', 'property_type'], as_index=False)['price_aprox_usd'].agg({np.mean, len}, fill_values=0),2)
df_grouped = pd.DataFrame(dfz.to_records())
df_grouped.columns = ['state_name', 'property_type' , 'quantity', 'price_mean']
df_grouped
data_type_ph_mask = df_grouped.property_type == "PH"
data_type_ph = df_grouped.loc[data_type_ph_mask, :]
data_type_apartment_mask = df_grouped.property_type == "apartment"
data_type_apartment = df_grouped.loc[data_type_apartment_mask, :]
data_type_house_mask = df_grouped.property_type == "house"
data_type_house = df_grouped.loc[data_type_house_mask, :]
data_type_store_mask = df_grouped.property_type == "store"
data_type_store = df_grouped.loc[data_type_store_mask, :]
data_type_ph
g = sns.jointplot("price_mean", "quantity", data_type_ph, kind='kde')
g = sns.jointplot("price_mean", "quantity", data_type_apartment, kind='kde')
g = sns.jointplot("price_mean", "quantity", data_type_house, kind='kde')
g = sns.jointplot("price_mean", "quantity", data_type_store, kind='kde')